In [1]:
import os
import numpy as np
import pandas as pd
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt

from sklearn.metrics import mean_squared_error
from tensorflow.keras.optimizers import Adam
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
from tensorflow.keras.callbacks import EarlyStopping

import warnings
warnings.filterwarnings('ignore')

In [2]:
find_word = "sc_ap_stock"

df = pd.read_excel('../../../data/tb_stock.xlsx')
df = df[['sc_date', find_word]]

df['sc_date']         = pd.to_datetime(df['sc_date'])         # datetime 형식으로 변환
df.dropna(inplace=True)                                       # 결측치가 있는 확인 및 제거

scaler                = MinMaxScaler(feature_range=(0, 1))    # 정규화
scaled_data           = scaler.fit_transform(df[[find_word]])

train_size            = int(len(scaled_data) * 0.8)           # 데이터셋 분리 (80% 학습용, 20%, 테스트용으로 사용)
train_data, test_data = scaled_data[:train_size], scaled_data[train_size:]

In [3]:
# LSTM 입력을 위해 데이터셋 재구성
def create_datest(dataset, look_back=60):
    X, y = [], []
    for i in range(len(dataset) - look_back -1):
        X.append(dataset[i:(i + look_back), 0])
        y.append(dataset[i + look_back, 0])
    return np.array(X), np.array(y)

look_back = 60
X_train, y_train = create_datest(train_data, look_back)
X_test, y_test   = create_datest(test_data, look_back)

# LSTM 입력을 위해 데이터 구조 변경 [samples, time steps, featrues]
X_train = np.reshape(X_train, (X_train.shape[0], X_train.shape[1], 1))
X_test  = np.reshape(X_test, (X_test.shape[0], X_test.shape[1], 1))

# LSTM 모델을 빌드하는 함수
def build_model(units=50, learning_rate=0.001):
    model = Sequential()
    model.add(LSTM(units=units, return_sequences=True, input_shape=(X_train.shape[1], 1)))
    model.add(LSTM(units=units, return_sequences=False))
    model.add(Dense(units=25))
    model.add(Dense(units=1))

    optimizer = Adam(learning_rate=learning_rate)
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    return model

In [4]:
# 세밀한 그리드 서치 파라미터 설정
param_grid = {
    'units': [30, 40, 50],
    'learning_rate': [0.001, 0.005, 0.01, 0.05],
    'batch_size': [16, 24, 32],
    'epochs': [15, 25, 35]
}
best_model = None
best_mse = float('inf')
best_params = {}
model_save_path = "best_model.h5"  # 모델을 저장할 경로

In [5]:
total_iterations = np.prod([len(v) for v in param_grid.values()])
pbar = tqdm(total=total_iterations, desc='Grid Search Progress', ncols=100, mininterval=1.0)

for units in param_grid['units']:
    for learning_rate in param_grid['learning_rate']:
        for batch_size in param_grid['batch_size']:
            for epochs in param_grid['epochs']:
                model = build_model(units=units, learning_rate=learning_rate)

                # EarlyStopping 콜백 설정
                early_stopping = EarlyStopping(
                    monitor='val_loss',
                    patience=5,
                    restore_best_weights=True,
                    verbose=0
                )

                history = model.fit(
                    X_train, y_train,
                    epochs=epochs,
                    batch_size=batch_size,
                    validation_data=(X_test, y_test),
                    callbacks=[early_stopping],
                    verbose=0
                )

                # 예측 및 평가
                y_pred = model.predict(X_test)
                mse = mean_squared_error(y_test, y_pred)

                # 과적합 여부 계산
                train_loss = history.history['loss'][-1]
                val_loss = history.history['val_loss'][-1]
                overfitting_ratio = val_loss / train_loss if train_loss > 0 else float('inf')

                # 과적합도 아니고 과소적합도 아닌 모델 중에서 MSE가 가장 낮은 모델 선택
                if 0.8 <= overfitting_ratio <= 1.2 and mse < best_mse:
                    best_mse = mse
                    best_model = model
                    best_params = {
                        'units': units,
                        'learning_rate': learning_rate,
                        'batch_size': batch_size,
                        'epochs': epochs,
                        'train_loss': train_loss,
                        'val_loss': val_loss,
                        'overfitting_ratio': overfitting_ratio
                    }

                    # 최적의 모델 저장
                    best_model.save(model_save_path)
                    print(f"\n최적의 모델이 {model_save_path}에 저장되었습니다.")
                    print("모델 파라미터:")
                    for key, value in best_params.items():
                        print(f"{key}: {value:.5f}" if isinstance(value, float) else f"{key}: {value}")

                pbar.update(1)

pbar.close()

# 최적의 모델에 대한 정보 출력X
print("\n최적의 모델 파라미터:")
for key, value in best_params.items():
    print(f"{key}: {value:.5f}" if isinstance(value, float) else f"{key}: {value}")

Grid Search Progress:   0%|                                                 | 0/108 [00:00<?, ?it/s]

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 24ms/step


Grid Search Progress:   1%|▎                                      | 1/108 [01:26<2:33:58, 86.34s/it]

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


Grid Search Progress:   2%|▋                                      | 2/108 [03:15<2:56:14, 99.76s/it]

...

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


Grid Search Progress:  99%|████████████████████████████████████▋| 107/108 [3:02:41<00:33, 33.10s/it]

21/21 ━━━━━━━━━━━━━━━━━━━━ 1s 22ms/step


Grid Search Progress: 100%|████████████████████████████████████| 108/108 [3:03:09<00:00, 101.76s/it]


최적의 모델 파라미터:
units: 40
learning_rate: 0.00500
batch_size: 16
epochs: 15
train_loss: 0.00012
val_loss: 0.00013
overfitting_ratio: 1.09538


In [7]:
# 최적의 파라미터 설정
units: 40
learning_rate: 0.00500
batch_size: 16
epochs: 15

# 모델 생성 및 컴파일
model = build_model(units=units, learning_rate=learning_rate)

# EarlyStopping 콜백 설정
early_stopping = EarlyStopping(
    monitor='val_loss',
    patience=5,
    restore_best_weights=True,
    verbose=1
)

# 모델 학습
history = model.fit(
    X_train, y_train,
    epochs=epochs,
    batch_size=batch_size,
    validation_data=(X_test, y_test),
    callbacks=[early_stopping],
    verbose=1
)

# 모델 저장
model_save_path = "best_model.h5"  # 저장 경로를 설정합니다.
model.save(model_save_path)
print(f"\n최적의 모델이 {model_save_path}에 저장되었습니다.")

Epoch 1/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 7s 25ms/step - loss: 0.0072 - val_loss: 6.6673e-04
Epoch 2/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 2.6501e-04 - val_loss: 6.6616e-04
Epoch 3/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 21ms/step - loss: 1.7925e-04 - val_loss: 4.0567e-04
Epoch 4/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.0237e-04 - val_loss: 8.5915e-04
Epoch 5/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.4636e-04 - val_loss: 4.2812e-04
Epoch 6/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 1.8654e-04 - val_loss: 6.3013e-04
Epoch 7/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 1.9701e-04 - val_loss: 6.4165e-04
Epoch 8/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 5s 24ms/step - loss: 1.1559e-04 - val_loss: 1.7268e-04
Epoch 9/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 23ms/step - loss: 2.2230e-04 - val_loss: 0.0016
Epoch 10/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - loss: 1.1426e-04 - val_loss: 1.3305e-04
Epoch 11/20
179/179 ━━━━━━━━━━━━━━━━━━━━ 4s 22ms/step - l


최적의 모델이 best_model.h5에 저장되었습니다.


In [10]:
# 과적합 여부 계산 및 출력
train_loss = best_params['train_loss']
val_loss = best_params['val_loss']

print(f"훈련 손실: {train_loss:.5f}")
print(f"검증 손실: {val_loss:.5f}")

if val_loss > train_loss:
    overfitting_ratio = val_loss / train_loss
    print(f"과적합 비율: {overfitting_ratio:.2f}")
    if overfitting_ratio > 1.2:
        print("경고: 모델이 과적합될 가능성이 있습니다.")
    else:
        print("모델이 과적합되지 않았습니다.")
else:
    print("모델이 과소적합되었거나, 잘 일반화되었습니다.")

KeyError: 'train_loss'

In [ ]:
def predict_future(model, data, days_ahead=30):
    future_predictions = []
    last_sequence = data[-look_back:]  # 가장 최근의 데이터 시퀀스

    for _ in range(days_ahead):
        # 최근 시퀀스로 예측 수행
        pred = model.predict(last_sequence.reshape(1, look_back, 1))
        future_predictions.append(pred[0, 0])
        # 시퀀스 업데이트
        last_sequence = np.append(last_sequence[1:], pred, axis=0)

    return np.array(future_predictions)

# 미래 30일 예측
days_to_predict = 30
future_preds = predict_future(best_model, scaled_data, days_ahead=days_to_predict)

# 스케일 복원
future_preds_rescaled = scaler.inverse_transform(future_preds.reshape(-1, 1))

# 미래 날짜 생성
last_date = df['sc_date'].iloc[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=days_to_predict)

# 결과 데이터프레임 생성
future_df = pd.DataFrame({
    'sc_date': future_dates,
    'Predicted_Price': future_preds_rescaled.flatten()
})

# 그래프 시각화
plt.figure(figsize=(16, 8))
plt.plot(df['sc_date'], df[find_word], label='Actual Price', color='blue')
plt.plot(future_df['sc_date'], future_df['Predicted_Price'], label='Predicted Future Price', color='red', linestyle='--')
plt.title(f'{find_word} Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
# 테스트 데이터에 대한 예측 수행
test_pred = best_model.predict(X_test)

# 예측값과 실제값을 원래 스케일로 복원
test_pred_rescaled = scaler.inverse_transform(test_pred)
y_test_rescaled = scaler.inverse_transform(y_test.reshape(-1, 1))

# 실제값과 예측값 비교 차트 그리기
plt.figure(figsize=(16, 8))

# 테스트 데이터에 대한 실제값 (녹색 실선)
plt.plot(df['sc_date'][train_size:train_size + len(y_test)], y_test_rescaled, label='Actual Test Price', color='green')

# 테스트 데이터에 대한 예측값 (주황색 점선)
plt.plot(df['sc_date'][train_size:train_size + len(y_test)], test_pred_rescaled, label='Predicted Test Price', color='orange', linestyle='--')

# 그래프 제목과 축 레이블 설정
plt.title(f'{find_word} Test Data Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()

In [ ]:
def predict_future(model, data, days_ahead=30):
    future_predictions = []
    last_sequence = data[-look_back:]  # 가장 최근의 데이터 시퀀스

    for _ in range(days_ahead):
        # 최근 시퀀스로 예측 수행
        pred = model.predict(last_sequence.reshape(1, look_back, 1))
        future_predictions.append(pred[0, 0])
        # 시퀀스 업데이트
        last_sequence = np.append(last_sequence[1:], pred, axis=0)

    return np.array(future_predictions)

# 미래 30일 예측
days_to_predict = 30
future_preds = predict_future(best_model, scaled_data, days_ahead=days_to_predict)

# 스케일 복원
future_preds_rescaled = scaler.inverse_transform(future_preds.reshape(-1, 1))

# 미래 날짜 생성
last_date = df['sc_date'].iloc[-1]
future_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=days_to_predict)

# 결과 데이터프레임 생성
future_df = pd.DataFrame({
    'sc_date': future_dates,
    'Predicted_Price': future_preds_rescaled.flatten()
})

plt.figure(figsize=(16, 8))
plt.plot(df['sc_date'], df[find_word], label='Actual Price', color='blue')
plt.plot(future_df['sc_date'], future_df['Predicted_Price'], label='Predicted Future Price', color='red', linestyle='--')
plt.title(f'{find_word} Price Prediction')
plt.xlabel('Date')
plt.ylabel('Price')
plt.legend()
plt.show()